In [79]:
#Import modules
import pandas as pd

#Import user modules
import ncl.sqlsnippets as snips

#Import env
from os import getenv
from dotenv import load_dotenv

In [80]:
#Import env

load_dotenv(override=True)

OUTPUTDIR = getenv("OUTPUTDIR")
UPLOADPATH = getenv("UPLOADPATH")

UPLOADTAB = getenv("UPLOADTAB")

SQL_DATABASE = getenv("SQL_DATABASE")
SQL_SCHEMA = getenv("SQL_SCHEMA")
SQL_ADDRESS = getenv("SQL_ADDRESS")

SQL_TABLE = f"wf_pwr_{UPLOADTAB.lower()}"

In [81]:
#Load data
df_data = pd.read_csv(f"{OUTPUTDIR}{UPLOADPATH}{UPLOADTAB}.csv")

In [82]:
#Get years in data file
def derrive_years(df):
    #Return all years present in the df file (might be multiple in the case of the historic file)
    years_arr =  df["fyear"].drop_duplicates().values
    #Add quotations to each array element as they need to be within quotations when used in the query string
    years_arr = ["'" + value + "'" for value in years_arr]
    return years_arr

#Construct delete query to remove old data 
#(Maybe in future replace with UPDATE statement but this is easier for now)
def build_delete_query(years):
    query = f"DELETE FROM {SQL_SCHEMA}.{SQL_TABLE} "\
            f"WHERE fyear IN ({', '.join(years)})"
    
    return query

#Delete existing data in table as we only want to overwrite some of the data
def delete_old_pwr(engine, df):
    years = derrive_years(df)

    delete_query = build_delete_query(years)

    snips.execute_query(engine, delete_query)

In [83]:
#Connect to db
conn_str = snips.get_connection_string(SQL_ADDRESS, SQL_DATABASE)

#Create connection
engine = snips.connect_to_sql(conn_str=conn_str)

In [84]:
#Check if table exists
if snips.table_exists(engine, SQL_TABLE, SQL_SCHEMA):
    #If it exists delete old data
    delete_old_pwr(engine, df_data)

In [85]:
#Upload the data
snips.upload_to_sql(df_data, engine, SQL_TABLE, SQL_SCHEMA, replace=False)